In [6]:
import os
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.tools import DuckDuckGoSearchResults

llm=ChatOpenAI(
    temperature=0.1,
    model_name="gpt-3.5-turbo-0125",
)

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for")

class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    Example query: Stock Market Symbol for Apple Company
    """
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchResults()
        return ddg.run(query)

class CompanyOverviewToolArgsSchema(BaseModel):
    symbol : str = Field(
        description="Stock symbol of the company. Example: AAPL, TSLA",
    )

class CompanyOverviewTool(BaseTool):
    name="CompanyOverview"
    description="""
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewToolArgsSchema] = CompanyOverviewToolArgsSchema

    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()

class CompanyIncomeStatementTool(BaseTool):
    name="CompanyIncomeStatement"
    description="""
    Use this to get the income statement of company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewToolArgsSchema] = CompanyOverviewToolArgsSchema

    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()["annualReports"]

class CompanyStockPerformanceTool(BaseTool):
    name="CompanyStockPerformance"
    description="""
    Use this to get the weekly performance fo a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewToolArgsSchema] = CompanyOverviewToolArgsSchema

    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools = [
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool(),
        CompanyStockPerformanceTool(),
    ],
)

prompt = "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment."

agent.invoke(prompt)




> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Stock Market Symbol for Cloudflare Company'}`


[snippet: In the latest market close, Cloudflare (NET) reached $89.52, with a +1.8% movement compared to the previous day. The stock's performance was ahead of the S&P 500's daily gain of 0.16%. Meanwhile ..., title: Cloudflare (NET) Beats Stock Market Upswing: What Investors ... - Nasdaq, link: https://www.nasdaq.com/articles/cloudflare-net-beats-stock-market-upswing-what-investors-need-know], [snippet: About NET. Cloudflare, Inc. operates as a cloud services provider that delivers a range of services to businesses worldwide. The company provides an integrated cloud-based security solution to secure a range of combination of platforms, including public cloud, private cloud, on-premise, software-as-a-service applications, and IoT devices; and website and application security products ..., title: Cloudflare, Inc. (NET) Stock Price, Quote & New

{'input': "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment.",
 'output': "### Cloudflare Inc. (NET) Financial Information:\n\n#### Company Overview:\n- **Description:** Cloudflare, Inc. operates a cloud platform that offers a range of network services to companies worldwide.\n- **Market Capitalization:** $29.92 Billion\n- **Revenue (TTM):** $1.48 Billion\n- **EPS (TTM):** -$0.30\n- **Profit Margin:** -6.9%\n- **Operating Margin (TTM):** -8.65%\n- **Return on Assets (TTM):** -3.59%\n- **Return on Equity (TTM):** -13.4%\n- **Analyst Target Price:** $92.14\n- **52-Week High:** $116\n- **52-Week Low:** $61.34\n\n#### Income Statement (Recent Fiscal Year):\n- **Total Revenue:** $1.30 Billion\n- **Gross Profit:** $989.74 Million\n- **Operating Income:** -$185.49 Million\n- **Net Income:** -$183.95 Million\n\n#### Stock Performance:\n- **50-Day Moving Average:** $83.83\